In [1]:
#Globale paramtere for niluAPI
import sys
import numpy as np
import pandas as pd
from pandasql import sqldf
import seaborn as sns
import matplotlib.pyplot as plt

base_url = "https://api.nilu.no/stats/day/"
from_date = "2010-04-02"
to_date = "2016-12-31"
radius=20

#Oslo kordinater
latitude= 59.9139
longitude=10.7522


In [2]:
#Funksjon som bruker generell nilu API funksjon for å hente ut den rå dataen med luftkvalitet og lagre den i en json fil
def get_raw_data_niluAPI(): 
    
    endpoint = f"{base_url}/{from_date}/{to_date}/{latitude}/{longitude}/{radius}"
    output_file =  "../../data/raw_data/raw_air_quality_nilu_oslo.json"


    sys.path.insert(0, '../../src')
    from niluAPI.data_niluAPI import fetch_raw_data_niluAPI, process_and_save_raw_data
    raw_data= fetch_raw_data_niluAPI(endpoint)
    processed_data = process_and_save_raw_data(raw_data, output_file)
    return processed_data

get_raw_data_niluAPI()


Gruppert data er lagret under ../../data/raw_data/raw_air_quality_nilu_oslo.json


In [3]:
#Funksjon som bruker generell nilu API funksjon for å hente ut den rå dataen med luftkvalitet og lagre den i en json fil
def get_raw_data_niluAPI(): 
    
    endpoint = f"{base_url}/{from_date}/{to_date}/{latitude}/{longitude}/{radius}"
    output_file =  "../../data/raw_data/raw_air_quality_nilu_oslo.json"


    sys.path.insert(0, '../../src')
    from niluAPI.data_niluAPI import fetch_raw_data_niluAPI, process_and_save_raw_data
    raw_data= fetch_raw_data_niluAPI(endpoint)
    processed_data = process_and_save_raw_data(raw_data, output_file)
    return processed_data

get_raw_data_niluAPI()


Gruppert data er lagret under ../../data/raw_data/raw_air_quality_nilu_oslo.json


In [4]:
def clean_raw_data():
    
    raw_data_file =  "../../data/raw_data/raw_air_quality_nilu_oslo.json"
    clean_data_file = "../../data/clean_data/niluAPI_clean_data.json"
    cols= ["Verdi_NO2", "Verdi_O3", "Verdi_SO2"]
    
    sys.path.insert(0, '../../src')
    from niluAPI.data_niluAPI import remove_outliers, interpolate_and_save_clean_data
    # Først fjern outliers fra rådataene
    pivot_df = remove_outliers(raw_data_file, cols)

    # Hvis dataen ble lest riktig, prosesser og lagre dataen
    if pivot_df is not None:
        interpolate_and_save_clean_data(pivot_df, clean_data_file, from_date, to_date)  
clean_raw_data()

Fjerning av outliers:
Outliers er mer enn 3 standardavvik unna gjennomsnittet

Verdi_NO2:
Fjernet 39 outliers
Standaravvik: 21.74
Gjennomsnitt: 32.25

Verdi_O3:
Fjernet 8 outliers
Standaravvik: 16.29
Gjennomsnitt: 42.08

Verdi_SO2:
Fjernet 27 outliers
Standaravvik: 1.17
Gjennomsnitt: 2.23


Interpolering av Nan-verdier:
Verdi_NO2: 233 verdier ble interpolert
Verdi_O3: 574 verdier ble interpolert
Verdi_SO2: 393 verdier ble interpolert

Gruppert data er lagret under ../../data/clean_data/niluAPI_clean_data.json


In [5]:
#Funskjon som skal nomalfordele skjevheten i dataene med yeo-johnson metoden
def fix_skewness_data_niluAPI():
    
    sys.path.insert(0, '../../src/')
    from niluAPI.data_niluAPI import analyse_and_fix_skewness
    clean_data_file = "../../data/clean_data/niluAPI_clean_data.json"
    analyze_data_file= "../../data/analyzed_data/niluAPI_analyzed_data.json"
    threshold = 1.0
    cols= ["Verdi_NO2", "Verdi_O3", "Verdi_SO2"]
    analyse_and_fix_skewness(clean_data_file, analyze_data_file, threshold, cols)

fix_skewness_data_niluAPI()


Skjevhet før transformasjon:
→ Verdi_NO2: 1.12
→ Verdi_O3: -0.18
→ Verdi_SO2: 0.76

Påfører Yeo-Johnson på kolonner med skjevhet > ±1.0

Skjevhet etter transformasjon:
→ Verdi_NO2: -0.01
→ Verdi_O3: -0.18
→ Verdi_SO2: 0.76

Gruppert data er lagret under ../../data/analyzed_data/niluAPI_analyzed_data.json


In [6]:
#måntlig statistikk

df = pd.read_json("../../data/clean_data/niluAPI_clean_data.json")
query = """
SELECT 
    strftime('%Y-%m', Dato) AS Måned, 
    AVG(Verdi_NO2) AS Snitt_NO2,
    AVG(Verdi_O3) AS Snitt_O3,
    AVG(Verdi_SO2) AS Snitt_SO2,
    COUNT(*) AS AntallDager
FROM df
GROUP BY Måned
ORDER BY Måned
"""

    # Execute the query
pysqldf = lambda q: sqldf(q, globals())
monthly_stats = pysqldf(query)


print(monthly_stats)

      Måned  Snitt_NO2   Snitt_O3  Snitt_SO2  AntallDager
0   2010-04  35.294109  64.302514   2.862069           29
1   2010-05  21.199597  59.643414   2.394220           31
2   2010-06  20.604992  62.172153   1.661528           30
3   2010-07  19.245623  59.183333   1.759274           31
4   2010-08  22.120556  49.400958   1.744296           31
..      ...        ...        ...        ...          ...
76  2016-08  15.866144  31.959334   1.179835           31
77  2016-09  11.270938  29.060571   1.426891           30
78  2016-10  18.103540  30.474370   1.923178           31
79  2016-11  38.514975  25.146216   2.166528           30
80  2016-12  46.175656   8.485714   2.040703           31

[81 rows x 5 columns]
